In [17]:
# 这是代码示例
import win32com.client as win32
import os

def add_comment_to_word(docx_path, target_text, comment_text, output_docx_path):
    # 检查文件是否存在
    if not os.path.isfile(docx_path):
        print(f"文件不存在: {docx_path}")
        return

    # 打开 Word 应用
    word_app = win32.gencache.EnsureDispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开 Word 文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 查找目标文本并添加批注
    for para in doc.Paragraphs:
        if target_text in para.Range.Text:
            # 在目标文本的位置添加批注
            range_to_comment = para.Range
            range_to_comment.Find.Text = target_text
            range_to_comment.Find.Execute()
            doc.Comments.Add(range_to_comment, comment_text)

    # 保存并关闭文档
    doc.SaveAs(os.path.abspath(output_docx_path))
    doc.Close()
    word_app.Quit()

# 使用函数添加批注
add_comment_to_word('new.docx', '你好', 'hello', 'updated_new_with_comments.docx')


In [19]:
# 首页日期格式检查
import re
import win32com.client as win32

def is_chinese_date_format(date_text):
    """
    判断日期文本是否符合中文日期格式（例如：二〇二四年九月）。
    :param date_text: 待检测的日期文本
    :return: 如果符合格式则返回True，否则返回False
    """
    pattern = r'[〇一二三四五六七八九十]{4}年[一二三四五六七八九十]{1,2}月'
    match = re.match(pattern, date_text)
    return match is not None

def mark_incorrect_date(paragraph, incorrect_text):
    start = paragraph.Range.Text.find(incorrect_text)
    end = start + len(incorrect_text)
    rng = paragraph.Range.Duplicate
    rng.Start += start
    rng.End = rng.Start + len(incorrect_text)
    rng.Font.Color = win32.constants.wdColorRed
    comment = paragraph.Range.Comments.Add(rng, "日期格式错误")
    return comment

def process_document(docx_path):
    """
    打开指定的 Word 文档，检查全文中的第一个日期格式，如果发现错误则进行标记和批注。
    """
    word_app = win32.gencache.EnsureDispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开 Word 文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))
    first_page = doc.Paragraphs

    for paragraph in first_page:
        text = paragraph.Range.Text.strip()
        # 查找段落中第一个符合日期格式的文本
        match = re.search(r'\d{4}年\d{1,2}月|\d{4}年[一二三四五六七八九十]{1,2}月|[〇一二三四五六七八九十]{4}年\d{1,2}月', text)
        if match:
            date = match.group(0)
            if not is_chinese_date_format(date):
                mark_incorrect_date(paragraph, date)
            break  # 找到第一个日期后跳出循环

    doc.SaveAs(os.path.abspath('test_processed.docx'))
    doc.Close()
    word_app.Quit()

process_document('test.docx')


In [8]:
import win32com.client as win32

def check_paragraph_format(paragraph,flag):
    format_issues = set()
    if flag:
        for char in paragraph.Range.Characters:
            font_name = char.Font.Name
            font_size = char.Font.Size
            text = char.Text

            if '\u4e00' <= text <= '\u9fff':  # 中文字符范围
                if font_name != '宋体':
                    format_issues.add("中文字体不是宋体")
                if font_size != 12:  # 小四对应的大小
                    format_issues.add("中文字体大小不是小四")

            # 判断英文字符和数字的字体
            elif text.isalpha() or text.isdigit():  # 英文或数字
                if font_name != 'Times New Roman':
                    format_issues.add("英文或数字字体不是Times New Roman")
                if font_size != 12:
                    format_issues.add("英文或数字字体大小不是小四")
    else:
        font_name = paragraph.Range.Font.Name
        font_size = paragraph.Range.Font.Size

        if font_name != '宋体':
           format_issues.add("字体不是宋体")
        if font_size != 12:
           format_issues.add("字体大小不是小四")
    # 检查段落的行距是否为1.5倍行距
    if paragraph.LineSpacingRule != win32.constants.wdLineSpace1pt5:
        format_issues.add("行距不为1.5倍")

    # 遍历段落中的每个字符，检查字体和字号
    return list(format_issues) if format_issues else None

def add_comment(paragraph, issues):
    """
    在段落中添加批注，内容为格式问题。
    """
    rng = paragraph.Range.Duplicate
    comment_text = "未满足的格式要求：" + "；".join(issues)
    paragraph.Range.Comments.Add(rng, comment_text)

def process_document(docx_path):
    """
    打开指定的 Word 文档，检测正文部分的格式，并对不符合要求的部分添加批注。
    """
    # 打开 Word 应用程序
    word_app = win32.Dispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 遍历文档的所有段落
    for paragraph in doc.Paragraphs:
        # 跳过空段落
        if not paragraph.Range.Text.strip():
            continue

        # 检查段落是否在表格内，或者是否在页眉、页脚中
        if (paragraph.Range.Information(12) or  # wdWithInTable = 12
            paragraph.Range.StoryType in [6, 7]):  # 使用常量值6和7替代wdHeaderStory和wdFooterStory
            continue  # 跳过表格、页眉和页脚

        # 进一步判断段落是否为正文（例如，基于样式判断）
        if paragraph.Style.NameLocal == '正文':
            # 检查段落格式
            issues = check_paragraph_format(paragraph,True)
            if issues:
                add_comment(paragraph, issues)


    # 保存文档
    doc.SaveAs(os.path.abspath('test_processed.docx'))

    # 关闭文档和应用程序
    doc.Close()
    word_app.Quit()

# 调用函数处理文档
process_document('test.docx')


AttributeError: wdLineSpace1pt5

In [11]:
import win32com.client as win32
import os

def check_paragraph_format(paragraph, flag):
    """
    检查段落中的文本格式是否符合要求：
    - 中文字体：宋体
    - 英文字体：Times New Roman
    - 字体大小：小四（12pt）
    - 行距：1.5倍
    如果不满足要求，则返回具体不满足的条件，否则返回None。
    """
    format_issues = set()

    if flag:
        # 遍历段落中的每个字符，检查字体和字号
        for char in paragraph.Range.Characters:
            font_name = char.Font.Name
            font_size = char.Font.Size
            text = char.Text

            if '\u4e00' <= text <= '\u9fff':  # 中文字符范围
                if font_name != '宋体':
                    format_issues.add("中文字体不是宋体")
                if font_size != 12:  # 小四对应的大小
                    format_issues.add("中文字体大小不是小四")

            # 判断英文字符和数字的字体
            elif text.isalpha() or text.isdigit():  # 英文或数字
                if font_name != 'Times New Roman':
                    format_issues.add("英文或数字字体不是Times New Roman")
                if font_size != 12:
                    format_issues.add("英文或数字字体大小不是小四")
    else:
        font_name = paragraph.Range.Font.Name
        font_size = paragraph.Range.Font.Size

        if font_name != '宋体':
           format_issues.add("字体不是宋体")
        if font_size != 12:
           format_issues.add("字体大小不是小四")

    # 检查段落的行距是否为1.5倍行距
    if paragraph.LineSpacingRule != win32.constants.wdLineSpace1pt5:
        format_issues.add("行距不为1.5倍")

    return list(format_issues) if format_issues else None

def add_comment(paragraph, issues):
    """
    在段落中添加批注，内容为格式问题。
    """
    rng = paragraph.Range.Duplicate
    comment_text = "未满足的格式要求：" + "；".join(issues)
    paragraph.Range.Comments.Add(rng, comment_text)

def process_document(docx_path):
    """
    打开指定的 Word 文档，检测正文部分的格式，并对不符合要求的部分添加批注。
    """
    # 打开 Word 应用程序
    word_app = win32.Dispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 遍历文档的所有段落
    for paragraph in doc.Paragraphs:
        # 跳过空段落
        if not paragraph.Range.Text.strip():
            continue

        # 检查段落是否在表格内，或者是否在页眉、页脚中
        if (paragraph.Range.Information(12) or  # wdWithInTable = 12
            paragraph.Range.StoryType in [6, 7]):  # 使用常量值6和7替代wdHeaderStory和wdFooterStory
            continue  # 跳过表格、页眉和页脚

        # 进一步判断段落是否为正文（例如，基于样式判断）
        if paragraph.Style.NameLocal == '正文':
            # 检查段落格式
            issues = check_paragraph_format(paragraph, True)
            if issues:
                add_comment(paragraph, issues)

    # 保存文档
    doc.SaveAs(os.path.abspath('test_processed.docx'))

    # 关闭文档和应用程序
    doc.Close()
    word_app.Quit()

# 调用函数处理文档
process_document('test.docx')


AttributeError: wdLineSpace1pt5

In [1]:
import win32com.client as win32
import os

def check_paragraph_format(paragraph):
    format_issues = []

    # 检查段落的行距是否为1.5倍行距
    if paragraph.LineSpacingRule != win32.constants.wdLineSpace1pt5:
        format_issues.append("行距不为1.5倍")

    # 检查段落的字体和字号
    font_name = paragraph.Range.Font.Name
    font_size = paragraph.Range.Font.Size

    # 判断字体是否为宋体
    if font_name != '宋体':
        format_issues.append("字体不是宋体")

    # 判断字体大小是否为小四（12pt）
    if font_size != 12:
        format_issues.append("字体大小不是小四")

    return format_issues if format_issues else None

def add_comment(paragraph, issues):
    """
    在段落中添加批注，内容为格式问题。
    """
    rng = paragraph.Range.Duplicate
    comment_text = "目录格式问题：" + "；".join(issues)
    paragraph.Range.Comments.Add(rng, comment_text)

def process_document(docx_path):
    """
    打开指定的 Word 文档，检测目录部分的格式，并对不符合要求的部分添加批注。
    """
    # 打开 Word 应用程序
    word_app = win32.Dispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 检查文档是否包含目录
    if doc.TablesOfContents.Count > 0:
        toc = doc.TablesOfContents(1)  # 获取第一个目录
        # 遍历目录的所有段落
        for paragraph in toc.Range.Paragraphs:
            # 检查段落格式
            issues = check_paragraph_format(paragraph)
            if issues:
                # 在目录的第一行添加批注
                add_comment(paragraph, issues)
                break  # 只处理目录的第一行
    else:
        print("文档中未找到目录。")

    # 保存文档
    doc.SaveAs(os.path.abspath('test_processed.docx'))

    # 关闭文档和应用程序
    doc.Close()
    word_app.Quit()

# 调用函数处理文档
process_document('test.docx')


NameError: name 'os' is not defined

In [5]:
import win32com.client as win32
import os

def check_header_footer_format(header_footer):
    """
    检查页眉和页脚的格式是否符合要求：
    - 字体：宋体
    - 字体大小：五号（10.5pt）
    如果不满足要求，则返回具体不满足的条件，否则返回None。
    """
    format_issues = []

    # 检查页眉或页脚的字体和字号
    for paragraph in header_footer.Range.Paragraphs:
        font_name = paragraph.Range.Font.Name
        font_size = paragraph.Range.Font.Size

        # 判断字体是否为宋体
        if font_name != '宋体':
            format_issues.append("字体不是宋体")

        # 判断字体大小是否为五号（10.5pt）
        if font_size != 10.5:
            format_issues.append("字体大小不是五号")

    return format_issues if format_issues else None

def add_comment(range, issues):
    """
    在指定范围中添加批注，内容为格式问题。
    """
    comment_text = "格式问题：" + "；".join(issues)
    range.Comments.Add(range, comment_text)

def process_document(docx_path):
    """
    打开指定的 Word 文档，检测页眉页脚格式，并添加批注。
    """
    # 打开 Word 应用程序
    word_app = win32.Dispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 遍历每个节
    for section in doc.Sections:
        try:
            # 检查页脚
            primary_footer = section.Footers(win32.constants.wdHeaderFooterPrimary)
            if primary_footer:
                footer_issues = check_header_footer_format(primary_footer)
                if footer_issues:
                    add_comment(primary_footer.Range, footer_issues)

            # 检查页眉
            primary_header = section.Headers(win32.constants.wdHeaderFooterPrimary)
            if primary_header:
                header_issues = check_header_footer_format(primary_header)
                if header_issues:
                    add_comment(primary_header.Range, header_issues)

        except Exception as e:
            print(f"检查节中的页眉或页脚时发生错误: {e}")

        # 检查封面或扉页是否有页眉和页脚
        try:
            if (not section.Footers(win32.constants.wdHeaderFooterPrimary).Range.Text.strip() and
                not section.Headers(win32.constants.wdHeaderFooterPrimary).Range.Text.strip()):
                print("封面或扉页，通常没有页眉和页脚")
        except Exception as e:
            print(f"检查封面或扉页的页眉页脚时发生错误: {e}")

    # 保存文档
    doc.SaveAs(os.path.abspath('test_processed.docx'))

    # 关闭文档和应用程序
    doc.Close()
    word_app.Quit()

# 调用函数处理文档
process_document('test.docx')


检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary
检查节中的页眉或页脚时发生错误: wdHeaderFooterPrimary
检查封面或扉页的页眉页脚时发生错误: wdHeaderFooterPrimary


In [ ]:
import win32com.client as win32

def check_paragraph_format(paragraph):
    format_issues = []

    # 检查段落的行距是否为1.5倍行距
    if paragraph.LineSpacingRule != win32.constants.wdLineSpace1pt5:
        format_issues.append("行距不为1.5倍")

    # 检查段落的字体和字号
    font_name = paragraph.Range.Font.Name
    font_size = paragraph.Range.Font.Size

    # 判断字体是否为宋体
    if font_name != '宋体':
        format_issues.append("字体不是宋体")

    # 判断字体大小是否为小四（12pt）
    if font_size != 12:
        format_issues.append("字体大小不是小四")

    return format_issues if format_issues else None

def add_comment(paragraph, issues):
    """
    在段落中添加批注，内容为格式问题。
    """
    rng = paragraph.Range.Duplicate
    comment_text = "目录格式问题：" + "；".join(issues)
    paragraph.Range.Comments.Add(rng, comment_text)

def process_document(docx_path):
    """
    打开指定的 Word 文档，检测目录部分的格式，并对不符合要求的部分添加批注。
    """
    # 打开 Word 应用程序
    word_app = win32.Dispatch('Word.Application')
    word_app.Visible = False  # 不显示 Word 窗口

    # 打开文档
    doc = word_app.Documents.Open(os.path.abspath(docx_path))

    # 检查文档是否包含目录
    if doc.TablesOfContents.Count > 0:
        toc = doc.TablesOfContents(1)  # 获取第一个目录
        # 遍历目录的所有段落
        for paragraph in toc.Range.Paragraphs:
            # 检查段落格式
            issues = check_paragraph_format(paragraph)
            if issues:
                # 在目录的第一行添加批注
                add_comment(paragraph, issues)
                break  # 只处理目录的第一行
    else:
        print("文档中未找到目录。")

    # 保存文档
    doc.SaveAs(os.path.abspath('test_processed.docx'))

    # 关闭文档和应用程序
    doc.Close()
    word_app.Quit()

# 调用函数处理文档
process_document('test.docx')


In [14]:
import win32com.client as win32
import os

def check_table_paragraph_alignment(doc):
    for table in doc.Tables:
        if table.Rows.Count == 0:  # 检查是否为空表格
            continue
        
        # 找到表格前面的段落
        prev_paragraph = table.Range.Previous(win32.constants.wdParagraph)

        if prev_paragraph is None:
            continue

        paragraph_text = prev_paragraph.Range.Text.strip()

        # 如果内容为空或为 "设计文件分发表" 则跳过检查
        if not paragraph_text or "设计文件分发表" in paragraph_text:
            continue

        # 检查对齐方式，是否为左对齐
        if prev_paragraph.Alignment != win32.constants.wdAlignParagraphLeft:
            add_comment(prev_paragraph, "此段落未左对齐")

def add_comment(paragraph, comment_text):
    rng = paragraph.Range.Duplicate
    paragraph.Range.Comments.Add(rng, comment_text)

if __name__ == '__main__':
    word_app = win32.gencache.EnsureDispatch('Word.Application')
    word_app.Visible = False

    doc_path = 'test.docx'
    doc = word_app.Documents.Open(os.path.abspath(doc_path))

    check_table_paragraph_alignment(doc)

    doc.SaveAs(os.path.abspath('test_processed.docx'))
    doc.Close()
    word_app.Quit()


AttributeError: '<win32com.gen_py.Microsoft Word 12.0 Object Library.Range instance at 0x2205468729008>' object has no attribute 'Range'

In [16]:
# 表名格式检查
import win32com.client as win32
import os
import re

def check_table_paragraph_alignment(doc):
    for table in doc.Tables:
        if table.Rows.Count == 0:  # 检查是否为空表格
            continue
        
        # 找到表格前的段落
        prev_range = table.Range.Previous()  # 获取表格前的 Range
        if prev_range is None or prev_range.Paragraphs.Count == 0:
            continue

        prev_paragraph = prev_range.Paragraphs(prev_range.Paragraphs.Count)
        paragraph_text = prev_paragraph.Range.Text.strip()

        # 如果内容为空或为 "设计文件分发表" 则跳过检查
        if not paragraph_text or "设计文件分发表" in paragraph_text:
            continue

        # 检查对齐方式，是否为左对齐
        if prev_paragraph.Alignment != win32.constants.wdAlignParagraphLeft:
            add_comment(prev_paragraph, "此段落未左对齐")



def add_comment(paragraph, comment_text):
    """如果段落中没有批注，则添加批注"""
    if not any(comment.Range.Text == comment_text for comment in paragraph.Range.Comments):
        rng = paragraph.Range.Duplicate
        paragraph.Range.Comments.Add(rng, comment_text)

if __name__ == '__main__':
    word_app = win32.gencache.EnsureDispatch('Word.Application')
    word_app.Visible = False

    doc_path = 'test.docx'
    doc = word_app.Documents.Open(os.path.abspath(doc_path))

    check_table_paragraph_alignment(doc)

    doc.SaveAs(os.path.abspath('test_processed.docx'))
    doc.Close()
    word_app.Quit()
